In [5]:
import numpy as np
import pandas as pd
import seaborn as sn
from typing import Tuple
import time

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from feature_engine.selection import (
    DropDuplicateFeatures,
    DropConstantFeatures,
)

Примерно так будет выглядеть каждый из ваших датасетов для пользователя. Датасеты будет удобно объединять в классы для удобного использования

In [4]:
class DatasetName:
    train_data: pd.DataFrame
    test_data: pd.DataFrame
    
    def __init__(self, train_data: pd.DataFrame, test_data: pd.DataFrame):
        self.train_data = train_data
        self.test_data = test_data

    @classmethod
    def load(cls) :
        """
        loads datasets with given name
        :return: (train_dataframe, test_dataframe)
        """
        train = pd.read_csv("data/Santander_train.csv")
        return DatasetName(train, None)

    def prepare(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """
        does preprocessing with dataset, feature generation, etc.
        :return: (train_x, train_y, test_x, test_y)
        """
        TARGET = 'TARGET'
        y = self.train_data[TARGET]
        self.train_data.drop(labels=[TARGET, 'ID'], axis=1, inplace=True)
        
        x_train, x_test, y_train, y_test = train_test_split(self.train_data, y, test_size=0.3, random_state=1729, stratify=y)
        
        constant = DropConstantFeatures(tol=1)
        constant.fit(x_train)
        
        x_train = constant.transform(x_train)
        x_test = constant.transform(x_test)
        
        duplicates = DropDuplicateFeatures()
        duplicates.fit(x_train)
        
        x_train = duplicates.transform(x_train)
        x_test = duplicates.transform(x_test)
        
        return x_train, y_train, x_test, y_test

Вот пример того, как может выглядеть использование подобного интерфейса

In [3]:
x_train, y_train, x_test, y_test = DatasetName.load().prepare()